# Model calibration

Prepared by Omar A. Guerrero (oguerrero@turing.ac.uk, @guerrero_oa)

In this tutorial, we will calibrate the free parameters of PPI's model. First, we will load all the data that we have prepared in the previous tutorials. Then, we extract the relevant information and put it in adecquate data structures. Finally, we run the calibration function and save the results with the parameter values.

## Importing Python's libraries to manipulate and visualise data

In [2]:
import matplotlib.pyplot as plt
import pandas as pd

## Importing PPI's functions

In this example, we will import the PPI source code directly from the repository. This means that we will place a request to GitHub, download the `ppi.py` file, and copy it locally into the folder where these tutorials are saved. Then, we will import ppi.

In [4]:
import requests
url = 'https://raw.githubusercontent.com/oguerrer/ppi/main/source_code/ppi.py'
r = requests.get(url)
with open('ppi.py', 'w') as f:
    f.write(r.text)
import ppi

## Load data

### Indicators

In [7]:
df_indis = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_indicators.csv')

I0 = df_indis.I0.values
IF = df_indis.IF.values
successRate = df_indis.successRate.values


AttributeError: 'DataFrame' object has no attribute 'successRate'

### Interdependency network

In [ ]:
df_net = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_network.csv')


### Budget

In [ ]:
df_exp = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_expenditure.csv')


### Budget-indicator mapping

In [ ]:
df_rela = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_relational_table.csv')
